<a href="https://colab.research.google.com/github/Joshitha18/Language_Diarization/blob/main/AE_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp37-none-any.whl size=5888 sha256=8e2ab0baf375d46b3d10ab054c0a20965266c6a219cf6c013607fc5cec9d04c8
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [5]:
import python_speech_features
import numpy as np
import soundfile as sf
import pickle
import cv2
import glob

Step 1: Take 1000 files and find 200ms telugu segments in each

In [6]:
!gdown --id 1SzFGWDqv7_GNDl0x2nKT-cE0PRiPpoK7

Downloading...
From: https://drive.google.com/uc?id=1SzFGWDqv7_GNDl0x2nKT-cE0PRiPpoK7
To: /content/telugu_feat_1000.pkl
148MB [00:03, 37.2MB/s]


In [7]:
feat_file = open('telugu_feat_1000.pkl', "rb")
tel_feat = pickle.load(feat_file)

In [8]:
len(tel_feat[0][18])

39

Step 3

In [9]:
import numpy

In [10]:
for i in range(0,len(tel_feat)):
  tel_feat[i] = numpy.stack( tel_feat[i], axis=0 )

t = np.array(tel_feat[0:2467]) t.shape _: samples,19,39 samples: Number of 200 ms segments time steps: No of frames in it features : mfcc

In [11]:
#t.shape _: samples,19,39 
#samples: Number of 200 ms segments 
#time steps: No of frames in it 
#features : mfcc

In [12]:
t = np.array(tel_feat)

In [13]:
t.shape

(24678, 19, 39)

In [14]:
sequence = t

In [19]:
window_length = 19

In [20]:
train_size = 90
val_size = 10
train, val= sequence[:int(len(sequence)*train_size/100)],sequence[-int(len(sequence)*val_size/100):]

In [22]:
train.shape,val.shape

((22210, 19, 39), (2467, 19, 39))

In [23]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, LSTM, RepeatVector
from keras.models import Model
from keras.models import model_from_json
from keras import regularizers
import datetime
import time
import requests as req
import json
import pandas as pd
import pickle
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from keras import callbacks
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau

In [27]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

# define model
n_in = 19
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(19,39)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(39)))
model.compile(optimizer='adam', loss='mse')

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100)               56000     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 19, 100)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 19, 100)           80400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 19, 39)            3939      
Total params: 140,339
Trainable params: 140,339
Non-trainable params: 0
_________________________________________________________________


In [29]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", mode ="min", patience = 7)
model_save = ModelCheckpoint('wts_100.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_delta=0.1, mode='min')
model.fit(train, train, epochs=100, verbose=1, batch_size=457,validation_data=(val,val), callbacks=[earlystopping, model_save, reduce_lr_loss])

Epoch 1/100
49/49 [==============================] - 29s 233ms/step - loss: 204.7264 - val_loss: 82.8482
Epoch 2/100
49/49 [==============================] - 10s 210ms/step - loss: 79.6455 - val_loss: 71.8810
Epoch 3/100
49/49 [==============================] - 10s 210ms/step - loss: 69.1638 - val_loss: 65.6143
Epoch 4/100
49/49 [==============================] - 10s 208ms/step - loss: 62.7454 - val_loss: 62.0927
Epoch 5/100
49/49 [==============================] - 10s 212ms/step - loss: 58.3411 - val_loss: 59.6771
Epoch 6/100
49/49 [==============================] - 10s 209ms/step - loss: 55.1259 - val_loss: 57.2148
Epoch 7/100
49/49 [==============================] - 10s 209ms/step - loss: 52.2684 - val_loss: 55.2513
Epoch 8/100
49/49 [==============================] - 10s 210ms/step - loss: 50.4675 - val_loss: 53.2116
Epoch 9/100
49/49 [==============================] - 10s 210ms/step - loss: 48.9441 - val_loss: 51.8821
Epoch 10/100
49/49 [==============================] - 10s 211ms

In [30]:
pred = model.predict(sequence, batch_size=457)

In [31]:
sequence[0][0]

array([-8.28719032e+00, -1.13347611e+01, -3.01328985e+01, -1.02183549e+00,
        5.05976291e+00,  6.01580903e+00,  1.42289013e+01,  1.76303173e+01,
        4.21620537e+00,  5.38378425e+00, -2.38234858e+00,  6.46840059e+00,
        9.40192492e+00, -6.28889692e-02, -9.51486909e-01, -3.42003309e-01,
       -9.88968581e-01, -2.08766134e+00,  3.97873890e-01, -1.34747231e+00,
       -2.45847306e-01,  2.33128228e+00, -2.42901207e-01,  9.74727779e-01,
       -1.00072325e-01, -1.33486978e+00, -1.59122761e-02,  1.37564663e-01,
        1.48142018e-01,  4.96692185e-01,  3.04829931e-01, -3.96158790e-02,
        3.91048911e-01,  3.36084104e-01, -7.19189540e-01, -3.65012013e-02,
        4.59455374e-01,  1.00436816e-02, -6.19904972e-01])

In [32]:
pred[0][0]

array([-7.8742456e+00, -1.2392062e+01, -2.6106621e+01, -4.5868902e+00,
        3.1828597e+00,  8.4367619e+00,  8.7706194e+00,  1.6867088e+01,
        8.6274920e+00,  1.3488873e+00,  1.4838252e+00,  3.7194910e+00,
        1.4084139e+00, -1.6332927e-01, -1.1467078e+00,  1.4366051e+00,
        6.2197125e-01,  9.6522158e-01,  4.3573570e-01,  1.3509647e+00,
       -1.0408504e+00, -1.5879958e+00,  1.9652996e+00,  1.2567148e+00,
        8.3053881e-01,  5.5359018e-01,  2.5654811e-02,  9.5644444e-01,
        4.2731982e-01,  3.9970404e-01, -8.0796635e-01, -8.1574708e-01,
        4.8582770e-02, -1.2759534e+00, -1.2692261e+00, -6.1197579e-01,
       -4.8609752e-02,  9.4737127e-02, -3.0713820e-01], dtype=float32)

In [33]:
out = model.layers[0].output
m = Model(inputs = model.input, outputs=out)

In [35]:
train_output = m.predict(train)
val_output = m.predict(val)
output = m.predict(sequence)

In [36]:
len(output[0])

100